In [1]:
from conf import ONION_DIR
import os
import ast

import pandas as pd
from conf import DATA_DIR

In [2]:
track_dataframes = []
for subdir, dirs, files in os.walk(ONION_DIR):
    for file in files:
        track_dataframe = pd.read_csv(os.path.join(subdir, file), sep='\t')
        track_dataframes += [track_dataframe]

In [3]:
onion_to_compare = pd.concat(track_dataframes)
onion_to_compare['gems_profile'] = onion_to_compare['gems_profile'].apply(lambda x: ast.literal_eval(x))
onion_to_compare = onion_to_compare.set_index('id')
onion_to_compare = pd.merge(onion_to_compare, onion_to_compare['gems_profile'].apply(pd.Series).fillna(value=0.), right_index=True, left_index=True).reset_index(drop=False)
onion_to_compare = onion_to_compare.drop(columns=['gems', 'genre', 'album_name', 'popularity', 'gems_profile', 'popularity class'])

In [4]:
rename_columns = {
    'id': 'song_code',
    'microgenre': 'genre',
    'song': 'title',
    'tenderness': 'tend',
    'peacefulness': 'peac',
    'joyful activation': 'joya',
    'wonder': 'wond',
    'power': 'ener',
    'nostalgia': 'nost',
    'tension': 'tens',
    'sadness': 'sadn',
    'transcendence': 'tran',
}
onion_to_compare = onion_to_compare.rename(columns=rename_columns)

In [5]:
rename_microgenres = {
    'trance': 'trance',
    'soft rock': 'rock',
    'soul': 'soul',
    'smooth jazz': 'jazz',
    'indie pop': 'pop',
    'swing': 'swing',
    'pop punk': 'punk',
    'dubstep': 'dubstep',
    'europop': 'europop',
    'doom metal': 'metal',
    'funk': 'funk',
    'downtempo': 'downtempo',
    'ballad': 'ballad',
    'disco': 'disco',
    'bossa nova': 'nova',
}
onion_to_compare.genre = onion_to_compare.genre.map(rename_microgenres)

In [6]:
onion_to_compare = onion_to_compare[['song_code', 'wond', 'tran', 'tend', 'nost', 'peac', 'joya', 'ener',
       'sadn', 'tens', 'title', 'artist', 'genre']]

In [7]:
onion_to_compare.to_csv(DATA_DIR + 'onion_to_compare.csv', index=False)

In [18]:
gems_9 = ['wond', 'tran', 'tend', 'nost', 'peac', 'joya', 'ener', 'sadn', 'tens']
f = onion_to_compare[gems_9].to_numpy().flatten().reshape(-1, 1)

In [19]:
(f > 0.).sum() / len(f)

0.34854059357370615

In [10]:
threshold = 0.
onion_bin_to_compare = onion_to_compare.copy()
onion_bin_to_compare[gems_9] = (onion_bin_to_compare[gems_9] > threshold) * 1.
onion_bin_to_compare.to_csv(DATA_DIR + 'onion_bin_to_compare.csv', index=False)

In [11]:
onion_bin_to_compare

,song_code,wond,tran,tend,nost,peac,joya,ener,sadn,tens,title,artist,genre
0,wDGNxY33wVBqikW2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,Not On Drugs (Ali Payami Remix),Tove Lo,disco
1,a8uQsr6gnzSCNrFB,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Don't Let Me Be Misunderstood,Santa Esmeralda,disco
2,ZdoFOuRY0SRYxxuw,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,You Meant The World To Me,Taylor Dayne,disco
3,yQIJDbt3yfSoXAsU,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Scary But Fun,Grace Jones,disco
4,NFq9a3oLSVaGKJWx,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,I'm So Excited,The Pointer Sisters,disco
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,wb4LtQFBXxV8SaZp,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,I Follow Rivers,Lykke Li,pop
449,tqOQJQT0ziGdhRqD,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,Video Games,Lana Del Rey,pop
450,yfm7RER1PWTqgjIp,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Royals,Lorde,pop
451,xKv9A6zDQOEySeNZ,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Dog Days Are Over,Florence + the Machine,pop


Sparsity per emotion

In [20]:
f = onion_bin_to_compare[gems_9]
f.sum().sum() / len(f) * 9

0.34854059357370615